In [3]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import transformers
import torch
import oss

In [4]:
base_model = "YOUR_LLAMA_MODEL_PATH"
tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.19s/it]


In [5]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2
model.eval()
f = open('YOUR ITEM NAME FILE', 'r')
# the format of the item name file is 
# item_name item_id
# A 0
# B 1
lines = f.readlines()
f.close()
text = [_.split('\t')[0].strip(" ").strip('\"') for _ in lines] # remove the leading and trailing spaces and quotess make sure this preprocess is the same as the prediction
tokenizer.padding_side = "left"

['Xbox 360 Guitar Hero 5 Guitar Bundle',
 'Cake Mania 2 - Nintendo DS',
 'Ultimate I Spy - Nintendo Wii',
 'Call of Duty: Ghosts Prestige Edition - Xbox 360',
 'Call of Duty World at War Collector&#39;s Edition - Xbox 360']

In [6]:
from tqdm import tqdm
def batch(list, batch_size=1):
    chunk_size = (len(list) - 1) // batch_size + 1
    for i in range(chunk_size):
        yield list[batch_size * i: batch_size * (i + 1)]
item_embedding = []
for i, batch_input in tqdm(enumerate(batch(text, 16))):
    input = tokenizer(batch_input, return_tensors="pt", padding=True)
    input_ids = input.input_ids
    attention_mask = input.attention_mask
    outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
    hidden_states = outputs.hidden_states
    item_embedding.append(hidden_states[-1][:, -1, :].detach().cpu())
    # break
item_embedding = torch.cat(item_embedding, dim=0)
torch.save(item_embedding, 'item_embedding.pt')
    

1088it [05:35,  3.24it/s]


In [8]:
item_embedding.shape

torch.Size([17408, 4096])